# AlexNet CIFAR-10 Classifier



### Imports

In [1]:
import time
import torch
import numpy as np
from torch import nn
import torch.nn.functional as F
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

### Settings and Dataset

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Hyperparameters
random_seed = 1
learning_rate = 0.0001
num_epochs = 10
batch_size = 256
num_classes = 10
torch.manual_seed(random_seed)


train_transforms = transforms.Compose([transforms.Resize((70, 70)),
                                       transforms.RandomCrop((64, 64)),
                                       transforms.ToTensor()])

test_transforms = transforms.Compose([transforms.Resize((70, 70)),
                                      transforms.CenterCrop((64, 64)),
                                      transforms.ToTensor()])


train_dataset = datasets.CIFAR10(root='data',
                                     train=True,
                                     transform=train_transforms,
                                     download=True)

valid_dataset = datasets.CIFAR10(root='data',
                                     train=True,
                                     transform=test_transforms)
    
test_dataset = datasets.CIFAR10(root='data',
                                    train=False,
                                    transform=test_transforms)

train_loader = DataLoader(dataset=train_dataset,
                                  batch_size=batch_size)

test_loader = DataLoader(dataset=test_dataset,
                             batch_size=batch_size,
                             shuffle=False)


valid_loader = DataLoader(dataset=valid_dataset,
                                  batch_size=batch_size)


# Checking the dataset
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

Files already downloaded and verified
Image batch dimensions: torch.Size([256, 3, 64, 64])
Image label dimensions: torch.Size([256])


### Model

In [3]:
print('Training Set:\n')
for images, labels in train_loader:  
    print('Image batch dimensions:', images.size())
    print('Image label dimensions:', labels.size())
    print(labels[:10])
    break
    
# Checking the dataset
print('\nValidation Set:')
for images, labels in valid_loader:  
    print('Image batch dimensions:', images.size())
    print('Image label dimensions:', labels.size())
    print(labels[:10])
    break

# Checking the dataset
print('\nTesting Set:')
for images, labels in train_loader:  
    print('Image batch dimensions:', images.size())
    print('Image label dimensions:', labels.size())
    print(labels[:10])
    break


Training Set:

Image batch dimensions: torch.Size([256, 3, 64, 64])
Image label dimensions: torch.Size([256])
tensor([6, 9, 9, 4, 1, 1, 2, 7, 8, 3])

Validation Set:
Image batch dimensions: torch.Size([256, 3, 64, 64])
Image label dimensions: torch.Size([256])
tensor([6, 9, 9, 4, 1, 1, 2, 7, 8, 3])

Testing Set:
Image batch dimensions: torch.Size([256, 3, 64, 64])
Image label dimensions: torch.Size([256])
tensor([6, 9, 9, 4, 1, 1, 2, 7, 8, 3])


In [4]:
class AlexNet(nn.Module):

    def __init__(self, num_classes):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        logits = self.classifier(x)
        probas = F.softmax(logits, dim=1)
        return logits


In [5]:
model = AlexNet(num_classes)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

### Training

In [6]:
def compute_accuracy(model, data_loader, device):
    correct_pred, num_examples = 0, 0
    for i, (features, targets) in enumerate(data_loader):
        features = features.to(device)
        targets = targets.to(device)
        logits = model(features)
        _, predicted_labels = torch.max(logits, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100
    

for epoch in range(num_epochs):
    model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        features = features.to(device)
        targets = targets.to(device)
            
        # FORWARD AND BACK PROP
        logits = model(features)
        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()
        cost.backward()
        
        # UPDATE MODEL PARAMETERS
        optimizer.step()
        
        # LOGGING
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %04d/%04d | Cost: %.4f' 
                   %(epoch+1, num_epochs, batch_idx, 
                     len(train_loader), cost))

    model.eval()
    with torch.set_grad_enabled(False):
        print('Epoch: %03d/%03d | Train: %.3f%%' % (
              epoch+1, num_epochs, 
              compute_accuracy(model, train_loader, device=device)))

Epoch: 001/010 | Batch 0000/0196 | Cost: 2.3023
Epoch: 001/010 | Batch 0050/0196 | Cost: 2.0324
Epoch: 001/010 | Batch 0100/0196 | Cost: 1.8179
Epoch: 001/010 | Batch 0150/0196 | Cost: 1.7826
Epoch: 001/010 | Train: 34.622%
Epoch: 002/010 | Batch 0000/0196 | Cost: 1.6233
Epoch: 002/010 | Batch 0050/0196 | Cost: 1.5748
Epoch: 002/010 | Batch 0100/0196 | Cost: 1.5274
Epoch: 002/010 | Batch 0150/0196 | Cost: 1.4832
Epoch: 002/010 | Train: 44.014%
Epoch: 003/010 | Batch 0000/0196 | Cost: 1.4199
Epoch: 003/010 | Batch 0050/0196 | Cost: 1.3739
Epoch: 003/010 | Batch 0100/0196 | Cost: 1.2270
Epoch: 003/010 | Batch 0150/0196 | Cost: 1.3154
Epoch: 003/010 | Train: 51.762%
Epoch: 004/010 | Batch 0000/0196 | Cost: 1.3643
Epoch: 004/010 | Batch 0050/0196 | Cost: 1.2231
Epoch: 004/010 | Batch 0100/0196 | Cost: 1.1155
Epoch: 004/010 | Batch 0150/0196 | Cost: 1.1914
Epoch: 004/010 | Train: 57.084%
Epoch: 005/010 | Batch 0000/0196 | Cost: 1.1503
Epoch: 005/010 | Batch 0050/0196 | Cost: 1.1258
Epoch: 0

### Evaluation

In [7]:
with torch.set_grad_enabled(False):
    
    train_acc = compute_accuracy(model=model,
                                 data_loader=test_loader,
                                 device=device)
    
    test_acc = compute_accuracy(model=model,
                                data_loader=test_loader,
                                device=device)
    
    valid_acc = compute_accuracy(model=model,
                                 data_loader=valid_loader,
                                 device=device)
    

print(f'Train ACC: {valid_acc:.2f}%')
print(f'Validation ACC: {valid_acc:.2f}%')
print(f'Test ACC: {test_acc:.2f}%')

Train ACC: 73.68%
Validation ACC: 73.68%
Test ACC: 68.27%
